In [292]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short. 
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after. 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

In [293]:
#Load Data
df = pd.read_csv("data/Euro_Salary.csv")
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,24/11/2020 13:55:19,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [294]:
df[393:396]

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
393,24/11/2020 19:15:02,30.0,Male,Moscow,Software Engineer,5,0,Middle,C,C/C++,14712.0,0,30,Full-time employee,Unlimited contract,Russian,101-1000,Product
394,24/11/2020 19:15:49,33.0,Male,Berlin,Product Manager,5,5,Senior,NaN,Python,70000.0,800,30,Full-time employee,NaN,German,101-1000,Product
395,24/11/2020 19:19:30,35.0,Male,Berlin,QA Engineer,11,10,Senior,Java,"Python, Javascript / Typescript, .NET, Java / ...",74400.0,NaN,30,Full-time employee,Unlimited contract,English,101-1000,Product


In [295]:
#df.info()
df["Other technologies/programming languages you use often"].value_counts()

Other technologies/programming languages you use often
Javascript / Typescript                                                                   44
Python                                                                                    37
SQL                                                                                       31
AWS, Docker                                                                               16
Kotlin                                                                                    15
                                                                                          ..
Python, Ruby, Java / Scala, Go, Rust, AWS, Kubernetes, Docker, Terraform, helm, packer     1
Kotlin, SQL, AWS, Kubernetes, Docker                                                       1
Python, R, Go                                                                              1
C/C++, PHP, SQL                                                                            1
Python, AWS, Ku

In [296]:
df.head()

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
0,24/11/2020 11:14:15,26.0,Male,Munich,Software Engineer,5,3,Senior,TypeScript,"Kotlin, Javascript / Typescript",80000.0,5000,30,Full-time employee,Unlimited contract,English,51-100,Product
1,24/11/2020 11:14:16,26.0,Male,Berlin,Backend Developer,7,4,Senior,Ruby,NaN,80000.0,NaN,28,Full-time employee,Unlimited contract,English,101-1000,Product
2,24/11/2020 11:14:21,29.0,Male,Berlin,Software Engineer,12,6,Lead,Javascript / Typescript,"Javascript / Typescript, Docker",120000.0,120000,30,Self-employed (freelancer),Temporary contract,English,101-1000,Product
3,24/11/2020 11:15:24,28.0,Male,Berlin,Frontend Developer,4,1,Junior,Javascript,NaN,54000.0,NaN,24,Full-time employee,Unlimited contract,English,51-100,Startup
4,24/11/2020 11:15:46,37.0,Male,Berlin,Backend Developer,17,6,Senior,C# .NET,".NET, SQL, AWS, Docker",62000.0,NaN,29,Full-time employee,Unlimited contract,English,101-1000,Product


In [297]:
df.sample(20)

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
1178,19/12/2020 09:31:16,31.0,NaN,Berlin,ML Engineer,5,2.5,Middle,Python,"Java / Scala, AWS, Google Cloud, Kubernetes, D...",77500.0,78000,28,Full-time employee,Unlimited contract,English,1000+,Product
1179,19/12/2020 09:54:24,34.0,Male,Berlin,Backend Developer,12,4,Senior,Kotlin,"Go, Rust, Kubernetes",75000.0,7500,27,Full-time employee,Unlimited contract,English,1000+,Product
604,25/11/2020 14:14:02,41.0,Male,Berlin,Software Engineer,18,7,Lead,PHP,"SQL, Google Cloud, Kubernetes, Docker",100000.0,100000,28,Full-time employee,Unlimited contract,English,51-100,Product
396,24/11/2020 19:24:50,37.0,Male,Hamburg,Software Engineer,15,5,Senior,Java,"Python, Javascript / Typescript, Java / Scala,...",70000.0,0,25,Full-time employee,Temporary contract,English,11-50,Startup
222,24/11/2020 13:35:41,33.0,Male,Boeblingen,DevOps,14,4,Senior,Golang,"Python, Javascript / Typescript, SQL, Go, Perl...",70000.0,NaN,30,Full-time employee,Unlimited contract,English,101-1000,Product
716,25/11/2020 23:32:29,26.0,Male,London,DevOps,3,0,Middle,"networking, linux, automation, cloud","Python, AWS",37000.0,NaN,25,Full-time employee,Unlimited contract,English,51-100,Product
927,30/11/2020 11:06:44,25.0,Male,Stuttgart,Data Scientist,0,0,Junior,Python,"R, SQL, Hadoop Hive",58000.0,NaN,30,Full-time employee,Unlimited contract,German,1000+,Handel
349,24/11/2020 17:53:18,38.0,Male,Koblenz,Systemadministrator,20,2,Senior,NaN,NaN,44000.0,NaN,25,Full-time employee,Unlimited contract,German,11-50,Systemhaus
889,28/11/2020 13:49:27,30.0,Male,Berlin,Head of BI,7,7,Head,Python,"Python, SQL, AWS, Kubernetes, Docker",100000.0,NaN,30,Full-time employee,Unlimited contract,English,101-1000,Startup
638,25/11/2020 16:39:24,35.0,Male,Berlin,Software Engineer,10,6,Senior,php,"Python, PHP, Javascript / Typescript, SQL",78000.0,78000,27,Full-time employee,Unlimited contract,German,up to 10,Product


DATA PREP

In [298]:
#Here I am looking at which columns dont add up to the total rows. In this case if they don't have 1253 they are missing values
df.count()

Timestamp                                                 1253
Age                                                       1226
Gender                                                    1243
City                                                      1253
Position                                                  1247
Total years of experience                                 1237
Years of experience in Germany                            1221
Seniority level                                           1241
Your main technology / programming language               1126
Other technologies/programming languages you use often    1096
Yearly brutto salary (without bonus and stocks) in EUR    1253
Yearly bonus + stocks in EUR                               829
Number of vacation days                                   1185
Employment status                                         1236
Сontract duration                                         1224
Main language at work                                  

In [299]:
#I'm changing the columns from object to float. These columns are the ones that im filling in their missing values with .mean 
df['Total years of experience'] = pd.to_numeric(df['Total years of experience'], errors='coerce')
df['Years of experience in Germany'] = pd.to_numeric(df['Years of experience in Germany'], errors='coerce')
df['Number of vacation days'] = pd.to_numeric(df['Number of vacation days'], errors='coerce')
df['Yearly bonus + stocks in EUR'] = pd.to_numeric(df['Yearly bonus + stocks in EUR'], errors='coerce')

In [300]:
#Finding mean of values I have
mean_age = df['Age'].mean()

mean_experience_total = df['Total years of experience'].mean()

mean_experience_germany = df['Years of experience in Germany'].mean()

mean_vacation_days = df['Number of vacation days'].mean()

mean_bonus_stocks = df['Yearly bonus + stocks in EUR'].mean()

#Filling in missing values in each column with their respective mean
df['Total years of experience'] = df['Total years of experience'].fillna(mean_experience_total)

df['Years of experience in Germany'] = df['Years of experience in Germany'].fillna(mean_experience_germany)

df['Number of vacation days'] = df['Number of vacation days'].fillna(mean_vacation_days)

df['Age'] = df['Age'].fillna(mean_age)

df['Yearly bonus + stocks in EUR'] = df['Yearly bonus + stocks in EUR'].fillna(mean_bonus_stocks)

In [301]:
#Inputing each column name here to put throw a loop
# Columns to fill with mode
columns_to_fill_mode = ['Gender', 'Position ', 'Seniority level', 'Employment status', 'Сontract duration', 'Company size',
                        'Company type', 'Main language at work', 'Your main technology / programming language','Other technologies/programming languages you use often']

# Fill missing values in each column with the mode. finding mode and filling in missing values
for column in columns_to_fill_mode:
    mode_value = df[column].mode()[0]  #
    df[column] = df[column].fillna(mode_value)  


In [302]:
# Convert the 'Yearly bonus + stocks in EUR' column to float
df['Yearly bonus + stocks in EUR'] = df['Yearly bonus + stocks in EUR'].astype(float)


In [303]:
#COnstructing new column that we will predict "Total Compensation". 
df["Total Compensation"] = (df["Yearly brutto salary (without bonus and stocks) in EUR"] + df["Yearly bonus + stocks in EUR"]).astype(float)

In [304]:
df.head()

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type,Total Compensation
0,24/11/2020 11:14:15,26.0,Male,Munich,Software Engineer,5.0,3.0,Senior,TypeScript,"Kotlin, Javascript / Typescript",80000.0,5.000000e+03,30.0,Full-time employee,Unlimited contract,English,51-100,Product,8.500000e+04
1,24/11/2020 11:14:16,26.0,Male,Berlin,Backend Developer,7.0,4.0,Senior,Ruby,Javascript / Typescript,80000.0,6.096870e+06,28.0,Full-time employee,Unlimited contract,English,101-1000,Product,6.176870e+06
2,24/11/2020 11:14:21,29.0,Male,Berlin,Software Engineer,12.0,6.0,Lead,Javascript / Typescript,"Javascript / Typescript, Docker",120000.0,1.200000e+05,30.0,Self-employed (freelancer),Temporary contract,English,101-1000,Product,2.400000e+05
3,24/11/2020 11:15:24,28.0,Male,Berlin,Frontend Developer,4.0,1.0,Junior,Javascript,Javascript / Typescript,54000.0,6.096870e+06,24.0,Full-time employee,Unlimited contract,English,51-100,Startup,6.150870e+06
4,24/11/2020 11:15:46,37.0,Male,Berlin,Backend Developer,17.0,6.0,Senior,C# .NET,".NET, SQL, AWS, Docker",62000.0,6.096870e+06,29.0,Full-time employee,Unlimited contract,English,101-1000,Product,6.158870e+06


In [305]:
df.count()

Timestamp                                                 1253
Age                                                       1253
Gender                                                    1253
City                                                      1253
Position                                                  1253
Total years of experience                                 1253
Years of experience in Germany                            1253
Seniority level                                           1253
Your main technology / programming language               1253
Other technologies/programming languages you use often    1253
Yearly brutto salary (without bonus and stocks) in EUR    1253
Yearly bonus + stocks in EUR                              1253
Number of vacation days                                   1253
Employment status                                         1253
Сontract duration                                         1253
Main language at work                                  

Building Model

In [306]:
# Check the types of all columns
all_column_types = df.dtypes
print(all_column_types)


Timestamp                                                  object
Age                                                       float64
Gender                                                     object
City                                                       object
Position                                                   object
Total years of experience                                 float64
Years of experience in Germany                            float64
Seniority level                                            object
Your main technology / programming language                object
Other technologies/programming languages you use often     object
Yearly brutto salary (without bonus and stocks) in EUR    float64
Yearly bonus + stocks in EUR                              float64
Number of vacation days                                   float64
Employment status                                          object
Сontract duration                                          object
Main langu

# Answers and Explainations
(Expand/modify as needed)

### Results

### Feature Selection Activities

### Hyperparameter Changes

In [307]:
numeric_features = ["Age", "Total years of experience", "Years of experience in Germany",
                    "Yearly brutto salary (without bonus and stocks) in EUR",
                    "Yearly bonus + stocks in EUR", "Number of vacation days"]
categorical_features = ["Gender", "City", "Position ", "Seniority level",
                        "Your main technology / programming language",
                        "Other technologies/programming languages you use often",
                        "Employment status", "Сontract duration", "Main language at work",
                        "Company size", "Company type"]


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  
    ('scaler', StandardScaler())  
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


model = RandomForestRegressor(n_estimators=100, random_state=42)


pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])


X = df.drop(columns=["Total Compensation"])
y = df["Total Compensation"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


pipeline.fit(X_train, y_train)


cvs = cross_val_score(pipeline, X_train, y_train, cv=5)
print('Training CrossVal Score:', cvs)
print("Average Train Score:", np.mean(cvs))


print('Testing score:', pipeline.score(X_test, y_test))


Training CrossVal Score: [ 9.99993560e-01 -3.35196094e+02  9.99988083e-01 -5.48777098e-03
  9.99453993e-01]
Average Train Score: -66.44042931609698
Testing score: 0.9996977387512865


In [308]:
model2 = SGDRegressor()


pipeline2 = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model2)])


X2 = df.drop(columns=["Total Compensation"])
y2 = df["Total Compensation"]


X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

# Fit the pipeline on the training data
pipeline2.fit(X_train2, y_train2)


cvs2 = cross_val_score(pipeline2, X_train2, y_train2, cv=5)
print('Training CrossVal Score:', cvs2)
print("Average Train Score:", np.mean(cvs2))


print('Testing score:', pipeline2.score(X_test2, y_test2))

Training CrossVal Score: [-0.73637824 -1.26511941 -1.55701181  0.99688583 -0.31520619]
Average Train Score: -0.5753659612482569
Testing score: 0.21902754380703116


c:\Users\2rory\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
